In [1]:
%pip install trino

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 12.5 MB/s eta 0:00:00 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
from trino.dbapi import connect

conn = connect(
    host="trino",
    port=8080,
    user="trino",
    catalog="memory",
    schema="default",
)
cur = conn.cursor()
cur.execute("SHOW SCHEMAS from memory")
rows = cur.fetchall()
print(rows);

[['default'], ['information_schema']]


Marquez -> http://localhost:3000/

In [3]:
cur.execute("""CREATE TABLE memory.default.trino_output (
                  ts_interval varchar,
                  max_log_date varchar,
                  max_log_time varchar,
                  max_ts varchar,
                  counter bigint,
                  user_id int,
                  max_price decimal(38, 18)
                )""");

### Start First flink job first to feed trino_output. Then proceed with other queries

In [4]:
cur.execute("""CREATE TABLE memory.default.user_features1 (user_id int, col_1 int,col_2 int, col_3 int)""");

In [5]:
cur.execute("""CREATE TABLE memory.default.user_features2 (user_id int, col_4 int,col_5 int, col_6 int)""");

In [6]:
cur.execute("""CREATE TABLE memory.default.user_features3 (user_id int, col_7 int,col_8 int, col_9 int)""");

In [7]:
cur.execute("""
   CREATE TABLE memory.default.trino_result
   AS SELECT
     r.user_id as user_id,
     sum(r.counter) as max_counter, 
     max(f1.col_1) as col_a, 
     max(f2.col_4) as col_b, 
     max(f3.col_7) as col_c
   FROM memory.default.trino_output r
   LEFT JOIN memory.default.user_features1 f1 ON f1.user_id=r.user_id
   LEFT JOIN memory.default.user_features2 f2 ON f2.user_id=r.user_id
   LEFT JOIN memory.default.user_features3 f3 ON f3.user_id=r.user_id
   GROUP BY r.user_id
""");

In [32]:
#cur.execute("""DROP TABLE memory.default.trino_result""")

In [8]:
cur.execute("select * from memory.default.trino_result");
rows = cur.fetchall()
print(rows);

[[104, 1, None, None, None], [103, 1, None, None, None], [102, 1, None, None, None], [101, 1, None, None, None], [105, 1, None, None, None]]
